# Build Azure OpenAI Client

In [2]:
# We will need this version of Azure OpenAI
%pip install openai==1.12.0

StatementMeta(, , , Waiting, )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 33.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 30.4 MB/s eta 0:00:00


  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-ca1ef571-6749-487d-ba6a-3c324b180622
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: openai
    Found existing installation: openai 0.27.8
    Not uninstalling openai at /home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-ca1ef571-6749-487d-ba6a-3c324b180622
    Can't uninstall 'openai'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.0.0 requires sentencepiece, which is not installed.
sentence-transformers 2.0.0 requires torchvision, which is n


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from openai import AzureOpenAI
import json

# Get keys and instantiate Azure OpenAI Client
with open('/lakehouse/default/Files/keys/aoai_key.json','r') as f:
    content = json.load(f)

ENDPOINT_URL = content.get('API_ENDPOINT')
API_KEY = content.get('API_KEY')
API_VERSION = '2023-09-01-preview'
MODEL_NAME = 'gpt-35-turbo'

client = AzureOpenAI(
    azure_endpoint=ENDPOINT_URL,
    api_key=API_KEY,
    api_version=API_VERSION,
)

StatementMeta(, , , Waiting, )

# Generate a Summary for each Book

We will use a reduction method leveraging our chunks to generate a summary for texts too long for gpt-35-turbo token limit

In [40]:
def generate_short_summary(text):

    prompt = f"""
        Summarize the following text in 2-3 sentences

        Text:
            {text}
    """

    try:
        completion = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {'role': 'system', 'content': 'Youre an AI assistant that summarizes text.'},
                {'role': 'user', 'content': prompt}
            ],
            temperature=0,
            max_tokens = 60
        )

        summary = completion.choices[0].message.content
        return summary

    except:
        return ''

StatementMeta(, , , Waiting, )

In [39]:
def summarize_json_chunks(json_doc):

    chunks = json_doc['chunks']
    text_chunks = [item['content'] for item in chunks]

    combined_summary = ''

    for text_chunk in text_chunks:
        summary = generate_short_summary(text_chunk)
        combined_summary += summary
    
    overall_summary = generate_short_summary(combined_summary)

    return overall_summary

StatementMeta(, , , Waiting, )

In [8]:
# Load and enrich JSON
import os
header_path = '/lakehouse/default/Files/book_enriched'

for file_name in os.listdir(header_path):
    file_path = os.path.join(header_path,file_name)

    # Open current file
    with open(file_path,'r') as f:
        json_content = f.read()
        json_doc = json.loads(json_content)
    
    summary = summarize_json_chunks(json_doc)

    enriched_json_doc = {
        'summary': summary
    }

    enriched_json_doc.update(json_doc)

    # Write AI enriched data back to source
    with open(file_path,'w') as f:
        json.dump(enriched_json_doc,f, indent=4)
    
    print(f'Enriched {file_name}')


StatementMeta(, , , Waiting, )

Enriched 104.json


Enriched 109.json


Enriched 117.json


Enriched 13.json


...

Enriched 61.json


Enriched 7.json


Enriched 71.json
Enriched 8.json


Enriched 9.json


Enriched 99.json


# Classify each Book Using the AOAI Generated Summary

In [ ]:
book_categories = [
    "Fantasy",
    "Science Fiction",
    "Mystery",
    "Romance",
    "History",
    "Interview",
    "Biography",
    "Classics",
    "Adventure",
    "Poetry",
    "Music",
    "Technology"
]

StatementMeta(, , , Waiting, )

In [ ]:
def classify_text(text):

    prompt = f"""
        Classify the following text into one of the categories

        Your response should look like:
            category
            
        Categories:
            {book_categories}

        Text:
            {text}
    """
    
    try:
        completion = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {'role': 'system', 'content': 'Youre an AI assistant that classifies text.'},
                {'role': 'user', 'content': prompt}
            ],
            temperature=0,
            max_tokens = 60
        )

        classification = completion.choices[0].message.content
        return classification

    except:
        return ''

StatementMeta(, , , Waiting, )

In [ ]:
# Load and enrich JSON
import os

enrich_json_docs = []
header_path = '/lakehouse/default/Files/book_enriched'

for file_name in os.listdir(header_path):
    file_path = os.path.join(header_path,file_name)

    # Open current file
    with open(file_path,'r') as f:
        json_content = f.read()
        json_doc = json.loads(json_content)
    
    summary = json_doc['summary']
    title = json_doc['title']
    category = classify_text(summary)

    enriched_json_doc = {
        'category': category
    }

    enriched_json_doc.update(json_doc)

    # Write AI enriched data back to source
    with open(file_path,'w') as f:
        json.dump(enriched_json_doc,f, indent=4)
    
    print(f'Enriched {file_name}')
    enrich_json_docs.append(enriched_json_doc)

StatementMeta(, , , Waiting, )

Enriched 61.json
Enriched 56.json


Enriched 3.json


Enriched 7.json
Enriched 5.json
Enriched 216.json


Enriched 235.json
Enriched 229.json
Enriched 41.json


...

Enriched 237.json


Enriched 104.json
Enriched 109.json
Enriched 117.json


Enriched 13.json
Enriched 136.json


Enriched 151.json
Enriched 156.json
Enriched 181.json


Enriched 2.json


# Join New Data to Table

In [ ]:
df = (
    spark.createDataFrame(enrich_json_docs)
    .select(['book_id','category','summary'])
)


df.createOrReplaceTempView('book_categories')

StatementMeta(, , , Waiting, )

In [ ]:
# Join df back to original table using SparkSQL
joined_df = spark.sql("""
    SELECT 
        B.*,
        BC.category,
        BC.summary
    FROM gutenberg_lakehouse.books B
    JOIN book_categories BC ON BC.book_id = B.book_id
""")

display(joined_df.limit(5))

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, d1d99bea-d7a7-4067-a942-530899224c1a)

In [ ]:
joined_df.write.format('delta').mode('overwrite').option('overwriteSchema','true').saveAsTable('books')

StatementMeta(, , , Waiting, )